# Bienvenue sur le Notebook Jupyter de la SAE 15 de Bryan LUNA et Elliot CAPITAINE

Nous expliquons les analyses petit à petit, à la fin du document nous démontrons si une corrélation a lieu.

Nous avions des bases en programmation notamment grâce à nos cours à l'IUT mais aussi aux cours du lycée mais nous n'avions jamais manipulé d'API, alors nous nous sommes entrainé avec les APIs en faisant d'abord un travaille de récupération des noms des parkings. On se sentait débordé par cette SAE alors nous avons fait le choix de faire un programme par mesure. Par exemple un programme pour les noms, un programme pour récupérer les données... Nous avons ensuite mis en relation les différents programmes entre eux.

### Programme 1 : Récupération des noms des parkings

In [ ]:
import requests
import json

url = "https://portail-api-data.montpellier3m.fr/offstreetparking?limit=1000"

# Faire une requête GET à l'API
response = requests.get(url)

# Vérifier si la requête a réussi (code de statut 200)
if response.status_code == 200:
    # Charger les données JSON
    data = response.json()

    # Extraire les noms des parkings
    parking_names = [parking['name']['value'] for parking in data]

    # Créer un dictionnaire avec les noms des parkings
    result = {"parking_names": parking_names}

    # Sauvegarder le dictionnaire dans un fichier JSON
    with open('parking_names.json', 'w', encoding='utf-8') as json_file:
        json.dump(result, json_file, ensure_ascii=False, indent=4)
else:
    print(f"La requête a échoué avec le code de statut {response.status_code}")


In [ ]:
import json

chemin_fichier_json = 'parking_names.json'

with open(chemin_fichier_json, 'r') as fichier_json:
    contenu_json = json.load(fichier_json)

print(json.dumps(contenu_json, indent=2))

Nous avons maintenant le nom des parkings de Montpellier en .json, nous pourrons nous en servir plus tard.

### Programme 2 : Récupération de l'historique du taux d'occupation de chaque parking.

Pour gagner du temps nous nous sommes réparti le travail. Un qui travaille sur les voitures et un autre sur les vélos.
Ici nous récupérons les données du taux d'occupation des parkings pendant tout le mois de février 2023 et ceci durant toute la journée.

In [ ]:
import requests
import json
import os
from datetime import datetime, timedelta

# Partie 1 - Récupération des données historiques

# Création du dossier "Données" s'il n'existe pas
folder_path = "Données_parking"
os.makedirs(folder_path, exist_ok=True)

# Initialisation du dictionnaire pour stocker toutes les données
all_data = {}

# API en temps réel des places de stationnement des vélos
realtime_url = "https://portail-api-data.montpellier3m.fr/offstreetparking?limit=1000"
response_realtime = requests.get(realtime_url)
realtime_data = response_realtime.json()

# Dates de début et de fin de la période demandée
start_date = datetime(2023, 2, 1)
end_date = datetime(2023, 2, 28)

# Récupération de l'historique pour les parkings de 1 à 23
for parking_id in range(1, 24):
    parking_url_template = "https://portail-api-data.montpellier3m.fr/parking_timeseries/urn%3Angsi-ld%3Aparking%3A{}/attrs/availableSpotNumber"

    # Récupération de l'historique pour le parking spécifique
    params = {"fromDate": start_date.isoformat(), "toDate": end_date.isoformat()}
    parking_url = parking_url_template.format(str(parking_id).zfill(3))
    response_parking = requests.get(parking_url, params=params)

    if response_parking.status_code == 200:
        parking_data = response_parking.json()

        # Ajout des données du parking actuel à la structure de données
        all_data[f"parking{parking_id}"] = parking_data

        # Enregistrement des données dans un fichier texte dans le dossier "Données"
        filename = os.path.join(folder_path, f"parking{parking_id}.json")
        with open(filename, "w") as file:
            json.dump(parking_data, file, indent=4)

        print(f"Historique du parking {parking_id} enregistré dans le fichier {filename}.")
    else:
        print(
            f"Erreur lors de la récupération de l'historique pour le parking {parking_id}. Code d'erreur : {response_parking.status_code}")

# Enregistrement des données de la structure complète dans un fichier "total.json"
total_filename = os.path.join(folder_path, "total.json")
with open(total_filename, "w") as total_file:
    json.dump(all_data, total_file, indent=4)

print(f"Données historiques enregistrées dans le fichier {total_filename}.")


### Programme 2 bis : Récupération de l'historique du taux d'occupation de chaque station de vélo.

Ici nous récupérons l'historique du taux d'occupation des stations de vélos sur le mois de Février.

<span style="color:red">ATTENTION CE PROGRAMME METS ENVIRON 3H30 POUR FINIR DE S'EXECUTER, J'AI MIS LE FICHIER AVEC LES DONNEES DEJA ENREGISTRE DANS LE DOSSIER POUR EVITER D'ATTENDRE</span>

In [ ]:
import requests
import json
import os
from datetime import datetime, timedelta

# Création du dossier "Données" s'il n'existe pas
folder_path = "Données"
os.makedirs(folder_path, exist_ok=True)

stations_url = "https://portail-api-data.montpellier3m.fr/bikestation?limit=1000"
timeseries_url_template = "https://portail-api-data.montpellier3m.fr/bikestation_timeseries/{}/attrs/availableBikeNumber"

# Récupération des données des stations
response_stations = requests.get(stations_url)
stations_data = response_stations.json()

# Dates de début et de fin
start_date = datetime(2023, 2, 1)
end_date = datetime(2023, 2, 15)

# Interval d'échantillonnage
sample_interval = timedelta(hours=2)

# Dictionnaire pour stocker les données par parking
parking_data = {}

# Récupération de l'historique pour les stations de 1 à 62
for i in range(1, 63):
    station_id = f"urn:ngsi-ld:station:{str(i).zfill(3)}"
    target_station = next((station for station in stations_data if station['id'] == station_id), None)

    if target_station:
        # Initialisation des données pour la station spécifique
        parking_data[station_id] = []

        # Récupération de l'historique jour par jour avec l'intervalle spécifié
        current_date = start_date
        while current_date <= end_date:
            params = {
                "fromDate": current_date.strftime("%Y-%m-%dT%H:%M:%S"),
                "toDate": (current_date + sample_interval).strftime("%Y-%m-%dT%H:%M:%S"),
            }
            timeseries_url = timeseries_url_template.format(station_id)
            response_timeseries = requests.get(timeseries_url, params=params)

            if response_timeseries.status_code == 200:
                timeseries_data = response_timeseries.json()
                parking_data[station_id].append({"date": current_date.isoformat(), "data": timeseries_data})
            else:
                print(f"Erreur lors de la récupération de l'historique pour la station {i}. Code d'erreur : {response_timeseries.status_code}")

            current_date += sample_interval

        # Enregistrement des données dans un fichier texte dans le dossier "Données"
        filename = os.path.join(folder_path, f"parking{i}.json")
        with open(filename, "w") as file:
            json.dump(parking_data[station_id], file, indent=4)

        print(f"Historique de la station {i} enregistré dans le fichier {filename}.")
    else:
        print(f"La station {i} n'a pas été trouvée dans les données des stations.")

# Enregistrement des données totales dans un fichier texte
total_filename = os.path.join(folder_path, "total.json")
with open(total_filename, "w") as total_file:
    json.dump(parking_data, total_file, indent=4)

print(f"Données totales enregistrées dans le fichier {total_filename}.")


### Programme 3 et 3 bis : Calcul de la moyenne d'occupation pour chaque parking et station de chaque jour

Nous faisons la moyenne d'occupation de tous les parkings et stations pour pouvoir s'en servir dans des tableaux avec des courbes

In [ ]:
import json
from datetime import datetime

def charger_noms_parkings(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data["parking_names"]

def calculer_moyenne_occupation_par_jour(file_path, noms_parkings):
    with open(file_path, 'r') as file:
        data = json.load(file)

    moyennes_par_jour = {}

    for parking_name, parking_data in data.items():
        # Utilisez le nom du parking à partir de la liste fournie
        parking_name = noms_parkings.pop(0)

        if 'index' in parking_data and 'values' in parking_data:
            for index, value in zip(parking_data['index'], parking_data['values']):
                jour = datetime.fromisoformat(index.split('.')[0]).date()
                key = (parking_name, jour)

                if key not in moyennes_par_jour:
                    moyennes_par_jour[key] = {'total_occupation': 0, 'total_entries': 0}

                moyennes_par_jour[key]['total_occupation'] += value
                moyennes_par_jour[key]['total_entries'] += 1

    moyennes_finales = {}

    for key, values in moyennes_par_jour.items():
        parking_name, jour = key
        moyenne = values['total_occupation'] / values['total_entries'] if values['total_entries'] > 0 else 0
        if parking_name not in moyennes_finales:
            moyennes_finales[parking_name] = {}
        moyennes_finales[parking_name][str(jour)] = moyenne

    return moyennes_finales

def enregistrer_resultats_dans_fichier(moyennes, nom_fichier):
    with open(nom_fichier, 'w') as file:
        for parking_name, moyennes_jour in moyennes.items():
            file.write(f"Park: {parking_name}\n")
            for jour, moyenne in moyennes_jour.items():
                file.write(f"   Jour: {jour}, Moyenne: {moyenne}\n")

chemin_fichier_noms_parkings = 'parking_names.json'
noms_parkings = charger_noms_parkings('parking_names.json')

chemin_fichier_json = 'Données_parking/total.json'
nom_fichier_resultats = 'resultats_moyennes.txt'

moyennes = calculer_moyenne_occupation_par_jour(chemin_fichier_json, noms_parkings)
enregistrer_resultats_dans_fichier(moyennes, nom_fichier_resultats)

print(f"Les résultats ont été enregistrés dans le fichier : {nom_fichier_resultats}")


In [3]:
import json
from datetime import datetime

def calculer_moyenne_occupation_par_jour(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)

    moyennes_par_jour = {}

    for parking_name, entries in data.items():
        for entry in entries:
            index = entry.get('data', {}).get('index')
            values = entry.get('data', {}).get('values')

            if index is not None and values is not None:
                for timestamp, value in zip(index, values):
                    jour = datetime.fromisoformat(timestamp.split('.')[0]).date()
                    key = (parking_name, jour)

                    if key not in moyennes_par_jour:
                        moyennes_par_jour[key] = {'total_occupation': 0, 'total_entries': 0}

                    moyennes_par_jour[key]['total_occupation'] += value
                    moyennes_par_jour[key]['total_entries'] += 1

    moyennes_finales = {}

    for key, values in moyennes_par_jour.items():
        parking_name, jour = key
        moyenne = values['total_occupation'] / values['total_entries'] if values['total_entries'] > 0 else 0
        if parking_name not in moyennes_finales:
            moyennes_finales[parking_name] = {}
        moyennes_finales[parking_name][str(jour)] = moyenne

    return moyennes_finales

def enregistrer_resultats_dans_fichier(moyennes, nom_fichier):
    with open(nom_fichier, 'w') as file:
        for parking_name, moyennes_jour in moyennes.items():
            file.write(f"Parc : {parking_name}\n")
            for jour, moyenne in moyennes_jour.items():
                file.write(f"   Jour : {jour}, Moyenne : {moyenne}\n")

chemin_fichier_json = 'Données_stations/total.json'
nom_fichier_resultats = 'resultats_moyennes_bikes.txt'

moyennes = calculer_moyenne_occupation_par_jour(chemin_fichier_json)
enregistrer_resultats_dans_fichier(moyennes, nom_fichier_resultats)

print(f"Les résultats ont été enregistrés dans le fichier : {nom_fichier_resultats}")


Les résultats ont été enregistrés dans le fichier : resultats_moyennes_bikes.txt


### Programme 4 : Génération des tableaux

Nous faisons apparaitre les courbes avec le taux d'occupation pour pouvoir voir l'évolution des parkings

In [ ]:
import json
import matplotlib.pyplot as plt
from datetime import datetime
from itertools import islice

# Charger les données des noms de parkings depuis le fichier JSON
with open('parking_names.json', 'r') as names_file:
    names_data = json.load(names_file)

# Charger les données des places disponibles depuis le fichier JSON
with open('Données_parking/total.json', 'r') as data_file:
    data = json.load(data_file)

# Associer les noms de parkings aux données
parkings_with_names = {names_data['parking_names'][i]: details for i, (_, details) in enumerate(data.items())}


# Fonction pour convertir les dates au format datetime
def convert_to_datetime(date_str):
    return datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%S.%f%z")


# Fonction pour générer le graphique
def generate_subplot(ax, parkings, title):
    ax.set_title(title)

    for parking, details in parkings.items():
        index = [convert_to_datetime(date) for date in details['index']]
        values = details['values']
        ax.plot(index, values, label=parking)

    ax.set_xlabel('Date et heure')
    ax.set_ylabel('Nombre de places disponibles')
    ax.legend()
    ax.grid(True)


# Générer les graphiques sur la même page
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 12))

# Diviser les parkings en deux groupes
half_parkings = len(parkings_with_names) // 2
first_half = dict(islice(parkings_with_names.items(), half_parkings))
second_half = dict(islice(parkings_with_names.items(), half_parkings, None))

generate_subplot(ax1, first_half, 'Première moitié des parkings')
generate_subplot(ax2, second_half, 'Deuxième moitié des parkings')

plt.tight_layout()
plt.show()


Nous avons ici nos courbes, on peut voir que les parkings suivent la même évolution ce qui est étonnant, nous pensions l'inverse. Aucun parking n'est saturé, sur le premier tableau on a l'impression mais en réalité c'est parce qu'il manque des données dans les historiques ou bien le parking devait être fermé. On peut voir qu'il y a des jours avec plus d'affluence que d'autres. Les pakings sont très occupé le durant le week-end. Les familles sortent, les jeunes aussi, Montpellier est une ville qui vie toute l'année et tous les jours mais surtout durant le week-end.

In [ ]:
import json
import matplotlib.pyplot as plt


def plot_station_curves(station_data):
    dates = station_data["data"]["index"]
    values = station_data["data"]["values"]

    plt.plot(dates, values)


def plot_grouped_stations(stations_data):
    num_stations = len(stations_data)

    for i in range(0, num_stations, 10):
        plt.figure(figsize=(10, 6))
        for j in range(i, min(i + 10, num_stations)):
            station_data = stations_data[j]
            plot_station_curves(station_data)
            plt.title(f"Group {i // 10 + 1}")
        plt.xlabel("Date")
        plt.ylabel("Taux d'occupation")
        plt.legend([f"Station {j + 1}" for j in range(i, min(i + 10, num_stations))])
        plt.show()


def main():
    with open("Données_stations/total.json", "r") as file:
        data = json.load(file)

    for station_id, station_data in data.items():
        plot_grouped_stations(station_data)


if __name__ == "__main__":
    main()


Nous pouvons voir que les stations de vélos fonctionnent à peu près de la même manière que les parkings pour voitures.

Que ce soit pour les parkings de voitures ou de vélos, il y a des changements constants. On peut observer une corrélation entre ces deux types de parkings. À Montpellier, un tramway est maintenant gratuit pour les habitants. Nous en déduisons que les habitants se garent près de chez eux et à proximité du tramway pour ensuite l'emprunter. Ensuite, ils utilisent les vélos pour se déplacer plus facilement, mais attention, ils ne redéposent pas le vélo au même endroit.

Il faudrait mener des analyses beaucoup plus approfondies afin de déterminer s'il existe une véritable corrélation. Nous ne savons pas si ce sont les mêmes personnes qui garent leur voiture et qui utilisent les vélos. Il pourrait tout aussi bien s'agir de touristes ou d'étudiants qui se servent uniquement des vélos.

Nos données ne peuvent pas être exploitées pour définir une corrélation.